In [ ]:
from flask import Flask, request, jsonify
import pandas as pd

app = Flask(__name__)

# --- Original SMA Function ---
def simple(df, window):
    sma_list = []

    for i in range(len(df)):
        if i < window - 1:
            sma_list.append(None)
        else:
            total = 0
            for j in range(i - window + 1, i + 1):
                total += df['close'].iloc[j]
            sma_list.append(total / window)

    df[f"sma_{window}"] = sma_list
    return df


# ------------------ GET API ------------------
@app.route("/api/sma", methods=["GET"])
def get_sma():
    try:
        # Read parameters
        window = request.args.get("window", type=int)
        
        if not window:
            return jsonify({"error": "Please provide window parameter"}), 400

        # Static sample data (same as your code)
        data = {
            "Date": pd.date_range(start="2006-01-01", periods=10),
            "close": [100, 102, 103, 105, 110, 111, 107, 108, 109, 110]
        }

        df = pd.DataFrame(data)

        # Calculate SMA
        result_df = simple(df, window)

        # Convert Date to string for JSON
        result_df["Date"] = result_df["Date"].astype(str)

        return jsonify(result_df.to_dict(orient="records"))

    except Exception as e:
        return jsonify({"error": str(e)}), 500


if __name__ == "__main__":
    app.run(debug=True, use_reloader=False, port=5006)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5006/ (Press CTRL+C to quit)


In [ ]:
from flask import Flask, request, jsonify
import pandas as pd

app = Flask(__name__)

# Function for SMA Calculation
def simple(df, window):
    sma_list = []
    for i in range(len(df)):
        if i < window - 1:
            sma_list.append(None)
        else:
            total = 0
            for j in range(i - window + 1, i + 1):
                total += df['close'].iloc[j]
            sma_list.append(total / window)

    df[f"sma_{window}"] = sma_list
    return df


# ----------------- API ROUTE -----------------
@app.route("/sma", methods=["POST"])
def sma_api():
    try:
        data = request.get_json()

        close_prices = data.get("close")
        window = data.get("window")
        start_date = data.get("start_date", "2006-01-01")

        if not close_prices or not window:
            return jsonify({"error": "Send JSON: { 'close':[...], 'window':5 }"}), 400

        df = pd.DataFrame({
            "Date": pd.date_range(start=start_date, periods=len(close_prices)),
            "close": close_prices
        })

        result = simple(df, window)
        return jsonify(result.to_dict(orient="records"))

    except Exception as e:
        return jsonify({"error": str(e)}), 500


if __name__ == "__main__":
    app.run(debug=True, use_reloader=False, port=5001)
